<a href="https://colab.research.google.com/github/hamzafarooq/multi-agent-course/blob/main/Module_5/Agents/Smol%20Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# smolagents

**smolagents** is a library that focuses on codeAgent, a kind of agent that performs “Actions” through code blocks, and then “Observes” results by executing the code.

You can also check this [blog post](https://huggingface.co/blog/smolagents) or the library’s [repo](https://github.com/huggingface/smolagents) in GitHub.

[HF smolagents](https://huggingface.co/docs/smolagents/en/index)

[Agents Course](https://huggingface.co/learn/agents-course/en/unit1/tutorial)

In [ ]:
!pip install 'smolagents[litellm]' --quiet
# !pip install smolagents --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.2 MB/s eta 0:00:00


**Building your agent**

To initialize a minimal agent, you need at least these two arguments:

**model**, a text-generation model to power your agent - because the agent is different from a simple LLM, it is a application code that uses a LLM as its brain. You can use any of these options:

**tools**, a list of Tools that the agent can use to solve the task. It can be an empty list. You can also add the default toolbox on top of your tools list by defining the optional argument add_base_tools=True.

Once you have these two arguments, tools and model, you can create an agent and run it. You can use any LLM you’d like, either through `HF Inference API`, `Local transformers Model`, `OpenAI or Anthropics API using LiteLLMModel` `ollama`, `Azure OpenAI`, or `mlx-lm`.

## Code Agent
The CodeAgent is our default agent. It will write and execute python code snippets at each step.

In [ ]:
# !pip install smolagents[litellm]
import os
from google.colab import userdata
from smolagents import CodeAgent, LiteLLMModel

model = LiteLLMModel(model_id="gpt-4o-2024-08-06", api_key=userdata.get('OPENAI_API_KEY')) # Could use 'gpt-4o'
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the 118th number in the Fibonacci sequence?                                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-2024-08-06 ──────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n):                                                                                                
      a, b = 0, 1                                                                                                  
      for _ in range(n):                                                                                           
          a, b = b, a + b                                                                                          
      return a                                                                                                     
                                                                                                                   
  fibonacci_118 = fibonacci(118)                                                                                   
  final_answer(fibonacci_118)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2046711111473984623691759

[Step 1: Duration 5.67 seconds| Input tokens: 2,047 | Output tokens: 98]

2046711111473984623691759

## Json Agent

We also support the widely-used way of writing actions as JSON-like blobs: this is ToolCallingAgent, it works much in the same way like CodeAgent, of course without additional_authorized_imports since it doesn’t execute code:

In [ ]:
from smolagents import ToolCallingAgent

agent = ToolCallingAgent(tools=[], model=model)
agent.run("Could you give me the 118th number in the Fibonacci sequence?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you give me the 118th number in the Fibonacci sequence?                                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-2024-08-06 ──────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The 118th number in the Fibonacci sequence is          │
│ 5358359254990966640871840.'}                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The 118th number in the Fibonacci sequence is 5358359254990966640871840.

[Step 1: Duration 1.44 seconds| Input tokens: 850 | Output tokens: 35]

'The 118th number in the Fibonacci sequence is 5358359254990966640871840.'

### Default toolbox


smolagents comes with a default toolbox for empowering agents, that you can add to your agent upon initialization with argument add_base_tools = True:

**DuckDuckGo** web search*: performs a web search using DuckDuckGo browser.

**Python code interpreter**: runs your LLM generated Python code in a secure environment. This tool will only be added to ToolCallingAgent if you initialize it with add_base_tools=True, since code-based agent can already natively execute Python code

**Transcriber**: a speech-to-text pipeline built on Whisper-Turbo that transcribes an audio to text.

In [ ]:
# from smolagents import DuckDuckGoSearchTool

# search_tool = DuckDuckGoSearchTool()
# print(search_tool("Who's the current president of USA?"))

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())

agent.run("Tell me updates about AI in 2025")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me updates about AI in 2025                                                                                │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  ai_2025_updates = web_search(query="AI updates 2025")                                                            
  print(ai_2025_updates)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[The 10 Biggest AI Trends Of 2025 Everyone Must Be Ready For Today - 
Forbes](https://www.forbes.com/sites/bernardmarr/2024/09/24/the-10-biggest-ai-trends-of-2025-everyone-must-be-ready
-for-today/)
Discover the 10 major AI trends set to reshape 2025: from augmented working and real-time decision-making to 
advanced AI legislation and sustainable AI initiatives.

[2025 in artificial intelligence - Wikipedia](https://en.wikipedia.org/wiki/2025_in_artificial_intelligence)
February 3 - OpenAI releases ChatGPT Deep Research, an artificial intelligence system integrated into ChatGPT, [7] 
which generates cited reports on a user-specified topic by autonomously browsing the web for 5 to 30 minutes. 
[8]February 6 - Mistral AI releases Le Chat, an AI assistant able to answer up to 1,000 words per second. 
[9]February 10 AI Action Summit takes place in Paris, France ...

[The year of AI clarity: 10 AI Forecasts for 2025 - 
Diplo](https://www.diplomacy.edu/blog/10-forecasts-for-ai-and-digitalisation-in-2025/)
Latest updates (January - March 2025) Recent AI Predictions and Their Realism. Geoffrey Hinton's Prediction on 
Human Extinction: Hinton estimated a 10-20% chance that AI could lead to human extinction within the next 30 years.
While this highlights the importance of AI safety, such predictions are speculative and reflect broader concerns 
rather than imminent realities.

[Five Trends in AI and Data Science for 2025 - MIT Sloan Management 
Review](https://sloanreview.mit.edu/article/five-trends-in-ai-and-data-science-for-2025/)
Here are the 2025 AI trends on our radar screens that leaders should understand and monitor. 1. Leaders will 
grapple with both the promise and hype around agentic AI. Let's get agentic AI — the kind of AI that does tasks 
independently — out of the way first: It's a sure bet for 2025's "most trending AI trend." Agentic AI seems to ...

[The State of AI in 2025: Key Takeaways from Stanford's ... - 
Unite.AI](https://www.unite.ai/the-state-of-ai-in-2025-key-takeaways-from-stanfords-latest-ai-index-report/)
The Stanford Institute for Human-Centered AI (HAI) has been tracking AI's growth and challenges through its annual 
AI Index Report, offering a comprehensive and data-driven overview. In its eighth edition for 2025, the report 
provides critical insights into the rapid advancements in AI, including breakthroughs in research, expanding 
real-world ...

[What's next for AI in 2025 - MIT Technology 
Review](https://www.technologyreview.com/2025/01/08/1109188/whats-next-for-ai-in-2025/)
In 2025, these trends will continue to be a boon for defense-tech companies like Palantir, Anduril, and others, 
which are now capitalizing on classified military data to train AI models.

[Top 10 AI Predictions for 2025](https://aimagazine.com/articles/top-10-ai-predictions-for-2025)
With this in mind, we examine the top 10 AI predictions for 2025 that can help you get ahead in the year to come. 
10. AI in Healthcare. AI in healthcare. ... The technology can automatically scan systems for vulnerabilities, 
recommend prioritised security updates, and even generate informed responses aligned with technical logs and global
...

[The State of AI 2025: 12 Eye-Opening Graphs - IEEE Spectrum](https://spectrum.ieee.org/ai-index-2025)
Explore the 2025 AI Index from Stanford University's Institute for Human-Centered Artificial Intelligence. These 12
charts reveal key trends, costs, and impacts of AI in 2025.

[AI Index 2025: State of AI in 10 Charts | Stanford 
HAI](https://hai.stanford.edu/news/ai-index-2025-state-of-ai-in-10-charts)
Small models get better, regulation moves to the states, and more.

[5 Predictions for AI in 2025 - TIME](https://time.com/7204665/ai-predictions-2025/)
In 2025, we'll begin to see a shift from chatbots and image generators toward "agentic" systems that can act 
autonomously to complete tasks, rather than simply answer questions, says AI ...

Out: None

[Step 1: Duration 1.20 seconds| Input tokens: 2,048 | Output tokens: 90]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Extracting key trends from the search results                                                                  
  trends = [                                                                                                       
      "Augmented working and real-time decision-making",                                                           
      "Advanced AI legislation and sustainable AI initiatives",                                                    
      "Agentic AI (AI that does tasks independently)",                                                             
      "Breakthroughs in research and real-world applications",                                                     
      "AI in healthcare",                                                                                          
      "Autonomous AI systems",                                                                                     
      "AI cybersecurity applications",                                                                             
      "Regulation moving to states",                                                                               
      "Small models getting better",                                                                               
      "Shift from chatbots and image generators to autonomous systems"                                             
  ]                                                                                                                
                                                                                                                   
  # Summarizing the updates                                                                                        
  summary = "AI in 2025 will focus on the following trends:\n"                                                     
  summary += "\n".join(f"- {trend}" for trend in trends)                                                           
  final_answer(summary)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: AI in 2025 will focus on the following trends:
- Augmented working and real-time decision-making
- Advanced AI legislation and sustainable AI initiatives
- Agentic AI (AI that does tasks independently)
- Breakthroughs in research and real-world applications
- AI in healthcare
- Autonomous AI systems
- AI cybersecurity applications
- Regulation moving to states
- Small models getting better
- Shift from chatbots and image generators to autonomous systems

[Step 2: Duration 10.63 seconds| Input tokens: 5,340 | Output tokens: 283]

'AI in 2025 will focus on the following trends:\n- Augmented working and real-time decision-making\n- Advanced AI legislation and sustainable AI initiatives\n- Agentic AI (AI that does tasks independently)\n- Breakthroughs in research and real-world applications\n- AI in healthcare\n- Autonomous AI systems\n- AI cybersecurity applications\n- Regulation moving to states\n- Small models getting better\n- Shift from chatbots and image generators to autonomous systems'

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())

agent.run("How many seconds would it take for a leopard at full speed to run?")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How many seconds would it take for a leopard at full speed to run?                                              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Get the top speed of a leopard                                                                                 
  leopard_speed_info = web_search(query="leopard top speed")                                                       
  print(leopard_speed_info)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[How Fast Are Leopards? Top Speeds and How It Compares ... - AZ 
Animals](https://a-z-animals.com/blog/how-fast-leopards-top-speeds-how-compares-other-big-cats/)
Leopards can run up to 36 miles per hour, but are one of the slowest of the big cat species. Learn how their speed 
compares to cheetahs, tigers, lions, and humans, and what factors affect their anatomy and agility.

[How Fast Can a Leopard Run? It's Slower than Most of Its Prey](https://africafreak.com/how-fast-can-a-leopard-run)
African leopards have a top speed of 56-60 km/h (35-37 mph). That's significantly slower than cheetahs and lions, 
as well as being slower than a lot of its prey. But this top speed isn't the full story. The leopard is one of the 
greatest predators on the African savannah and speed is just one piece of its hunting prowess.

[Leopard - Facts, Diet, Habitat & Pictures on Animalia.bio](https://animalia.bio/leopard)
Basic facts about Leopard: lifespan, distribution and habitat map, lifestyle and social behavior, mating habits, 
diet and nutrition, population size and status. ... Top speed. 58. 36. km/h mph. km/h mph . Weight. 28-90. 
61.6-198. kg lbs. kg lbs . Height. 57-70. 22.4-27.6. cm inch. cm inch . Length. 90-190. ... Leopards are top 
predators of ...

[16 Leopard Facts You Should Know - Fact Animal](https://factanimal.com/leopard/)
Learn about the leopard, the smallest and most widely distributed big cat in Africa and Asia. Find out its top 
speed, habitat, diet, predators, conservation status and interesting facts.

[Unraveling the Speed of Leopards: How Fast Can They Actually 
Run?](https://startingrun.com/unraveling-the-speed-of-leopards-how-fast-can-they-actually-run/)
A leopard's speed varies across different life stages, with cubs being slow and careful learners, juveniles gaining
in speed but needing to master control, adults reaching top speed with strategic control, and elder leopards 
experiencing reduced speed due to age. ... Fast, agile, and powerful, the cat sprints, moving at breakneck speed. 
It ...

[How Fast Can a Leopard Run? A Fascinating Look at Their 
Speed](https://tiger-universe.com/blogs/tiger-blog/how-fast-can-a-leopard-run)
The nimble leopard, known for its remarkable leopard speed, shows how evolution shaped top predators. These animals
can dash at 36 mph , making them some of the most skilled hunters. Their fast pace and agility let them thrive in 
different places, from open savannas to thick forests.

[How fast can leopards run and how agile are they? - 
WildLifeFAQs](https://www.wildlifefaq.com/leopard-speed-and-agility/)
Learn how fast leopards can run, how they adapt to different habitats, and how they use their climbing skills to 
hunt and escape. Find out how Amur leopards, a subspecies of leopard, are one of the most swift and agile predators
in the world.

[How Fast Is A Leopard? | Bite-Sized Knowledge - UntamedAnimals](https://untamedanimals.com/how-fast-is-a-leopard/)
Cheetahs are famous for their top speed which is more than double that of a leopard. Estimates of their top speed 
range from 50 to 80 mph or 80 to 128 km/h. Although cheetahs are not part of the 5 "big cats", they are often 
compared to them. Cheetahs are the fastest land animals and are perfectly adapted for speed, including a light 
build ...

[How Fast Is A Leopard? Top Speed & Acceleration](https://scalecompanions.com/how-fast-is-a-leopard/)
Learn about the amazing speed of leopards—from top speed to acceleration rates and their incredible hunting 
prowess. Discover how fast these fierce predators can run and chase prey in this comprehensive guide. Top Speed 
Recorded Maximum

[How Fast Can a Leopard Run (Average Speed) - 
brotherspets.com](https://brotherspets.com/how-fast-can-a-leopard-run/)
Leopards are known for their speed and agility, and they are capable of reaching impressive top speeds when they 
need to. According to various sources, leopards can run at speeds ranging from 30-56 mph (4

[Step 1: Duration 7.81 seconds| Input tokens: 2,052 | Output tokens: 157]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Convert the average top speed of a leopard from mph to m/s                                                     
  leopard_speed_mph = 58                                                                                           
  leopard_speed_ms = leopard_speed_mph * 0.44704                                                                   
                                                                                                                   
  # Distance in meters                                                                                             
  distance_meter = 100                                                                                             
                                                                                                                   
  # Calculate the time it would take for a leopard to run the distance                                             
  time_seconds = distance_meter / leopard_speed_ms                                                                 
  final_answer(time_seconds)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 3.856786710438625

[Step 2: Duration 11.87 seconds| Input tokens: 5,363 | Output tokens: 353]

3.856786710438625

### Custom Tool

In [ ]:
from huggingface_hub import list_models

task = "text-classification"

most_downloaded_model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
print(most_downloaded_model.id)

distilbert/distilbert-base-uncased-finetuned-sst-2-english


In [ ]:
from smolagents import tool
from huggingface_hub import list_models

@tool
def model_download_tool(task: str) -> str:
    """
    This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub.
    It returns the name of the checkpoint.

    Args:
        task: The task for which to get the download count.
    """
    most_downloaded_model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
    return most_downloaded_model.id

In [ ]:
from smolagents import CodeAgent, HfApiModel
agent = CodeAgent(tools=[model_download_tool], model=HfApiModel())
agent.run(
    "Can you give me the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the model that has the most downloads in the 'text-to-video' task on the Hugging    │
│ Face Hub?                                                                                                       │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  top_model = model_download_tool(task="text-to-video")                                                            
  print(top_model)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Lightricks/LTX-Video

Out: None

[Step 1: Duration 3.19 seconds| Input tokens: 2,075 | Output tokens: 60]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  top_model = model_download_tool(task="text-to-video")                                                            
  if top_model is not None:                                                                                        
      final_answer(top_model)                                                                                      
  else:                                                                                                            
      final_answer("No model found for the 'text-to-video' task.")                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Lightricks/LTX-Video

[Step 3: Duration 5.06 seconds| Input tokens: 6,713 | Output tokens: 229]

'Lightricks/LTX-Video'

### Change Agent System Prompt Templates

In [ ]:
#print(agent.prompt_templates["system_prompt"])

In [ ]:
#agent.logs #it will be depreciated
#agent.memory.steps

# Travel Advisory Agent

In [ ]:
!pip install googlemaps==4.10.0 --quiet

  Preparing metadata (setup.py) ... done


In [ ]:
from typing import Optional
from smolagents import CodeAgent, HfApiModel, tool

@tool
def get_travel_duration(start_location: str, destination_location: str, transportation_mode: Optional[str] = None) -> str:
    """Gets the travel time between two places.

    Args:
        start_location: the place from which you start your ride
        destination_location: the place of arrival
        transportation_mode: The transportation mode, in 'driving', 'walking', 'bicycling', or 'transit'. Defaults to 'driving'.
    """
    import os   # All imports are placed within the function, to allow for sharing to Hub.
    import googlemaps
    from datetime import datetime

    gmaps = googlemaps.Client(userdata.get('GOOGLE_MAP_API'))

    if transportation_mode is None:
        transportation_mode = "driving"
    try:
        directions_result = gmaps.directions(
            start_location,
            destination_location,
            mode=transportation_mode,
            departure_time=datetime(2025, 6, 6, 11, 0), # At 11, date far in the future
        )
        if len(directions_result) == 0:
            return "No way found between these places with the required transportation mode."
        return directions_result[0]["legs"][0]["duration"]["text"]
    except Exception as e:
        print(e)
        return e

agent = CodeAgent(tools=[get_travel_duration], model=model, additional_authorized_imports=["datetime"])

agent.run("Can you give me a nice one-day trip around Paris with a few locations and the times? Could be in the city or outside, but should fit in one day. I'm travelling only with a rented bicycle.")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me a nice one-day trip around Paris with a few locations and the times? Could be in the city or    │
│ outside, but should fit in one day. I'm travelling only with a rented car.                                      │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-2024-08-06 ──────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Choose locations for a day trip                                                                                
  locations = [                                                                                                    
      "Eiffel Tower",           # Iconic symbol of Paris                                                           
      "Louvre Museum",          # Famous art museum                                                                
      "Notre-Dame Cathedral",   # Historic landmark                                                                
      "Palace of Versailles"    # Splendid palace just outside Paris                                               
  ]                                                                                                                
                                                                                                                   
  print("Selected locations for the trip:", locations)                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Selected locations for the trip: ['Eiffel Tower', 'Louvre Museum', 'Notre-Dame Cathedral', 'Palace of Versailles']

Out: None

[Step 1: Duration 9.64 seconds| Input tokens: 2,065 | Output tokens: 207]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  itinerary_summary = """                                                                                          
  Based on the selected locations and the travel times, here is a nice one-day itinerary for exploring Paris and   
  its surroundings by car:                                                                                         
                                                                                                                   
  1. **Eiffel Tower**:                                                                                             
     - Visit from **09:00** to **10:30**                                                                           
     - Iconic symbol of Paris, perfect start to your day.                                                          
                                                                                                                   
  2. **Travel to Louvre Museum**:                                                                                  
     - Duration: **9 mins**                                                                                        
                                                                                                                   
  3. **Louvre Museum**:                                                                                            
     - Visit from **10:39** to **12:39**                                                                           
     - Explore the world's most famous art collections.                                                            
                                                                                                                   
  4. **Travel to Notre-Dame Cathedral**:                                                                           
     - Duration: **6 mins**                                                                                        
                                                                                                                   
  5. **Notre-Dame Cathedral**:                                                                                     
     - Visit from **12:45** to **13:45**                                                                           
     - Visit this historic Gothic architectural masterpiece.                                                       
                                                                                                                   
  6. **Travel to Palace of Versailles**:                                                                           
     - Duration: **44 mins**                                                                                       
                                                                                                                   
  7. **Palace of Versailles**:                                                                                     
     - Visit from **14:29** to **16:59**                                                                           
     - Experience the opulence and grandeur of the French monarchy.                                                
  """                                                                                                              
                                                                                                                   
  final_answer(itinerary_summary)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
Based on the selected locations and the travel times, here is a nice one-day itinerary for exploring Paris and its 
surroundings by car:

1. **Eiffel Tower**: 
   - Visit from **09:00** to **10:30** 
   - Iconic symbol of Paris, perfect start to your day.

2. **Travel to Louvre Museum**: 
   - Duration: **9 mins**

3. **Louvre Museum**: 
   - Visit from **10:39** to **12:39** 
   - Explore the world's most famous art collections.

4. **Travel to Notre-Dame Cathedral**: 
   - Duration: **6 mins**

5. **Notre-Dame Cathedral**: 
   - Visit from **12:45** to **13:45** 
   - Visit this historic Gothic architectural masterpiece.

6. **Travel to Palace of Versailles**: 
   - Duration: **44 mins**

7. **Palace of Versailles**: 
   - Visit from **14:29** to **16:59** 
   - Experience the opulence and grandeur of the French monarchy.

[Step 5: Duration 8.10 seconds| Input tokens: 15,947 | Output tokens: 1,450]

"\nBased on the selected locations and the travel times, here is a nice one-day itinerary for exploring Paris and its surroundings by car:\n\n1. **Eiffel Tower**: \n   - Visit from **09:00** to **10:30** \n   - Iconic symbol of Paris, perfect start to your day.\n\n2. **Travel to Louvre Museum**: \n   - Duration: **9 mins**\n\n3. **Louvre Museum**: \n   - Visit from **10:39** to **12:39** \n   - Explore the world's most famous art collections.\n\n4. **Travel to Notre-Dame Cathedral**: \n   - Duration: **6 mins**\n\n5. **Notre-Dame Cathedral**: \n   - Visit from **12:45** to **13:45** \n   - Visit this historic Gothic architectural masterpiece.\n\n6. **Travel to Palace of Versailles**: \n   - Duration: **44 mins**\n\n7. **Palace of Versailles**: \n   - Visit from **14:29** to **16:59** \n   - Experience the opulence and grandeur of the French monarchy.\n"